In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

from ezaggrid import AgGrid, get_license

In [3]:
def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product([mklbl('A', 4),
                                      mklbl('B', 2),
                                      mklbl('C', 4),
                                      mklbl('D', 2)],
                                    names=['RowIdx-1', 'RowIdx-2', 'RowIdx-3', 'RowIdx-4'])
index  =['-'.join(col).strip() for col in miindex.values]
micolumns = pd.MultiIndex.from_tuples([('a', 'foo', 'zap'),
                                       ('a', 'foo', 'zip'),
                                       ('a', 'bar', 'zap'),
                                       ('a', 'bar', 'zip'),
                                       ('b', 'foo', 'zap'),
                                       ('b', 'foo', 'zep'),
                                       ('b', 'bah', 'zep'),
                                       ('b', 'bah', 'zyp'),
                                       ('b', 'bah', 'zap'),
                                      ],
                                      names=['ColIdx-{}'.format(i) for i in range(1, 4)])
cols  =['-'.join(col).strip() for col in micolumns.values]
data = np.arange(len(miindex) * len(micolumns), dtype=np.float).reshape((len(miindex),len(micolumns)))
data = data.tolist()


dfrc = pd.DataFrame(data, index=miindex, columns=micolumns).sort_index().sort_index(axis=1)

dfr  = pd.DataFrame(data, index=miindex, columns=cols).sort_index().sort_index(axis=1)
dfr.columns.name = 'UniqueCol'

dfc  = pd.DataFrame(data, index=index, columns=micolumns).sort_index().sort_index(axis=1)
dfc.index.name = 'UniqueRow'

df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df.index.name = 'UniqueRow'
df.columns.name = 'UniqueCol'

dfrc.info()
dfr.info()
dfc.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 64 entries, (A0, B0, C0, D0) to (A3, B1, C3, D1)
Data columns (total 9 columns):
(a, bar, zap)    64 non-null float64
(a, bar, zip)    64 non-null float64
(a, foo, zap)    64 non-null float64
(a, foo, zip)    64 non-null float64
(b, bah, zap)    64 non-null float64
(b, bah, zep)    64 non-null float64
(b, bah, zyp)    64 non-null float64
(b, foo, zap)    64 non-null float64
(b, foo, zep)    64 non-null float64
dtypes: float64(9)
memory usage: 5.1+ KB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 64 entries, (A0, B0, C0, D0) to (A3, B1, C3, D1)
Data columns (total 9 columns):
a-bar-zap    64 non-null float64
a-bar-zip    64 non-null float64
a-foo-zap    64 non-null float64
a-foo-zip    64 non-null float64
b-bah-zap    64 non-null float64
b-bah-zep    64 non-null float64
b-bah-zyp    64 non-null float64
b-foo-zap    64 non-null float64
b-foo-zep    64 non-null float64
dtypes: float64(9)
memory usage: 5.1+ KB
<class 'pandas.core.frame.

In [4]:
df.head(10)

UniqueCol,a-foo-zap,a-foo-zip,a-bar-zap,a-bar-zip,b-foo-zap,b-foo-zep,b-bah-zep,b-bah-zyp,b-bah-zap
UniqueRow,,,,,,,,,
A0-B0-C0-D0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
A0-B0-C0-D1,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0
A0-B0-C1-D0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0
A0-B0-C1-D1,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0
A0-B0-C2-D0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0
A0-B0-C2-D1,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0
A0-B0-C3-D0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0
A0-B0-C3-D1,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0
A0-B1-C0-D0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0


In [5]:
css_rules = """
.number-cell {
    text-align: left;
}
.nb-even {
    color: red;
}
.nb-odd {
    color: blue;
}
"""

ccf = "function(params) { return params.value % 2 == 0 ? 'nb-even' : 'nb-odd'; }"
column_defs_1 = [{'field': df.index.name}] + [{'field': c, 'cellClass': ccf} for c in df.columns[:5]]

ccf = None
column_defs_2 = [{'field': df.index.name}] + [{'field': c, 'cellClass': ccf} for c in df.columns[:5]]

grid_options = {
    'columnDefs' : column_defs_1,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'onGridReady': """function(event){
        helpers.sizeToFit(event)
        // helpers.autoSizeAll(event)
    }
    """
}

grid_options_2 = {
    'columnDefs' : column_defs_2,
    'enableSorting': False,
    'enableFilter': True,
    'enableColResize': True,
    'onGridReady': """function(event){
        helpers.sizeToFit(event)
        // helpers.autoSizeAll(event)
    }
    """
}

for compressed in [True, False]:
    tag = 'compressed_json_in_base64' if compressed else 'plain_json'
    print(tag)
    ag = AgGrid(grid_data=df,
    #             grid_options=grid_options,
                grid_options_multi=[('opt1', grid_options), ('opt2', grid_options_2)],
                css_rules=css_rules,
                width=900,
                height=400,
                quick_filter=True,
                export_csv=True,
                export_excel=True,
                implicit_col_defs=True,
                index=True,
                keep_multiindex=False,
                theme='ag-theme-fresh',
                compress_data=compressed,
                license=get_license())
    with open('saved_aggrid_html_{}.html'.format(tag), 'w') as f:
        f.write(ag.html)
    ag.show()

compressed_json_in_base64


plain_json
